<div style="background-color:#C0392B; text-align: center;color:#19180F; font-size:40px; font-family:Arial; padding:10px; border: 5px solid #19180F; border-radius:10px">
Build a Text Multi-Class Classification Model using BERT
 </div>

<div style="background-color:#D98880; color:#19180F; text-align: center;font-size:30px; font-family:Arial; padding:10px; border: 5px solid #19180F; border-radius:10px"> Introduction & Motivation </div>
<div style="background-color:#D5D9F2; color:#19180F; font-size:20px; font-family:verdana; padding:10px; border: 5px solid #19180F; border-radius:10px "> 
In this notebook we are going to present our DCGAN. Its purpose is to generate
fake images that look like real images, after training on a particular dataset. 
We were interested in GANs because we  thought it would be really interesting to 
dive into the details of training one. For other types of deep learning 
architectures, it can be pretty straightforward to train a network, but that is 
not the case with GANs.
</div>



<div style="background-color:#D5D9F2; color:#19180F; font-size:20px; font-family:verdana; padding:10px; border: 5px solid #19180F; border-radius:10px "> 
Our training was executed on NVIDIA's last generation GPUs, `A100`. because on CPU this took more then 5 hours, And this is for `128x128` images.
We will discuss about scaling up our GAN later.  
</div>




<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Importing modules
    </div>

In [1]:
import re
import torch
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch.nn as nn
from transformers import BertModel
from transformers import BertTokenizer
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

/home/elhabachi/miniconda3/envs/chatbot/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
label_col = "Product"
tokens_path = "Output/tokens.pkl"
labels_path = "Output/labels.pkl"
data_path = "Input/complaints.csv"
model_path = "Output/bert_pre_trained.pth"
text_col_name = "Consumer complaint narrative"
label_encoder_path = "Output/label_encoder.pkl"
product_map = {'Vehicle loan or lease': 'vehicle_loan',
               'Credit reporting, credit repair services, or other personal consumer reports': 'credit_report',
               'Credit card or prepaid card': 'card',
               'Money transfer, virtual currency, or money service': 'money_transfer',
               'virtual currency': 'money_transfer',
               'Mortgage': 'mortgage',
               'Payday loan, title loan, or personal loan': 'loan',
               'Debt collection': 'debt_collection',
               'Checking or savings account': 'savings_account',
               'Credit card': 'card',
               'Bank account or service': 'savings_account',
               'Credit reporting': 'credit_report',
               'Prepaid card': 'card',
               'Payday loan': 'loan',
               'Other financial service': 'others',
               'Virtual currency': 'money_transfer',
               'Student loan': 'loan',
               'Consumer Loan': 'loan',
               'Money transfers': 'money_transfer'}

In [3]:
def save_file(name, obj):
    """
    Function to save an object as pickle file
    """
    with open(name, 'wb') as f:
        pickle.dump(obj, f)


def load_file(name):
    """
    Function to load a pickle object
    """
    return pickle.load(open(name, "rb"))

In [11]:
data=pd.read_csv(data_path, delimiter=',', quotechar='"',  engine='python',on_bad_lines='skip')

In [12]:
data.dropna(subset=[text_col_name], inplace=True)

In [13]:
data.replace({label_col: product_map}, inplace=True)

### Encode labels

In [14]:
label_encoder = LabelEncoder()
label_encoder.fit(data[label_col])
labels = label_encoder.transform(data[label_col])

In [16]:
save_file(labels_path, labels)
save_file(label_encoder_path, label_encoder)

### Process the text column

In [17]:
input_text = list(data[text_col_name])